### ACTION RECOGNITION USING MP_CV_KERAS_TF_LSTM

####  INSTALL OF DEPENDENCIES

In [ ]:
pip install --upgrade pip

In [ ]:
!pip install tensorflow-gpu

In [ ]:
!pip install tensorboard

In [ ]:
!pip install tensorflow==2.12.0 mediapipe opencv-python pandas sklearn matplotlib scikit-learn

#### IMPORTATION DES PREMIERES LIB

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import time



#### DECLARATION MEDIAPIPE HOLISTIC , DRAWS UTILS AND SOME FONCTION

In [4]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [5]:
#Style Draw Utils fct
def drawing_spec_landmarks(image , results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(color=(41, 18, 219),thickness = 1,circle_radius= 1),
                              mp_drawing.DrawingSpec(color=(41, 18, 219),thickness = 1,circle_radius= 1)
                             )
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(212, 17, 17),thickness = 1,circle_radius= 1),
                              mp_drawing.DrawingSpec(color=(212, 17, 17),thickness = 1,circle_radius= 1)
                             )
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(227, 16, 188),thickness = 1,circle_radius= 1),
                              mp_drawing.DrawingSpec(color=(227, 16, 188),thickness = 1,circle_radius= 1)
                             )
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(16,227,223),thickness = 1,circle_radius= 1),
                              mp_drawing.DrawingSpec(color=(16,227,223),thickness = 1,circle_radius= 1)
                             )

In [6]:
#MP detection fct
def mediapipe_detection(image , model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) #Conversion de l'image de BGR en RGB
    image.flags.writeable = False              #Modification de l'image desactiver
    results = model.process(image)            #Deection des landsmarks
    image.flags.writeable = True              #Modification de l'image activer
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #Convertion de l'image de BGR en RGB
    return image , results                    # Renvoi des resultats

In [7]:
# Test Capture fct :NB 'q' for close
cap = cv2.VideoCapture(0)
#Init holistic model
with mp_holistic.Holistic(min_detection_confidence = 0.5 ,min_tracking_confidence = 0.5 ) as holistic:
    
    while cap.isOpened() :
        ret,frame = cap.read()                    #Capture 

        #Detect Landmarks 

        image , results = mediapipe_detection(frame,holistic)
        
        #Draw 
        
        drawing_spec_landmarks(image,results)

        cv2.imshow("FRAME FEED ", image)          #Display

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [20]:
print(results.pose_landmarks)

landmark {
  x: 0.6062335
  y: 0.49561685
  z: -1.7284834
  visibility: 0.9930298
}
landmark {
  x: 0.65280193
  y: 0.39536256
  z: -1.7696213
  visibility: 0.9931023
}
landmark {
  x: 0.6812764
  y: 0.37896642
  z: -1.7696804
  visibility: 0.9930922
}
landmark {
  x: 0.70424
  y: 0.3637945
  z: -1.7698538
  visibility: 0.99240315
}
landmark {
  x: 0.5450904
  y: 0.40549907
  z: -1.7565572
  visibility: 0.9923803
}
landmark {
  x: 0.51559615
  y: 0.39720216
  z: -1.7567599
  visibility: 0.99150705
}
landmark {
  x: 0.49162096
  y: 0.38713992
  z: -1.7570289
  visibility: 0.99241036
}
landmark {
  x: 0.74579144
  y: 0.30896777
  z: -1.5834435
  visibility: 0.98957896
}
landmark {
  x: 0.4337291
  y: 0.3350695
  z: -1.5136678
  visibility: 0.9939666
}
landmark {
  x: 0.6568686
  y: 0.54254824
  z: -1.6187751
  visibility: 0.98147273
}
landmark {
  x: 0.5709215
  y: 0.5529639
  z: -1.5984724
  visibility: 0.98739237
}
landmark {
  x: 0.8557625
  y: 0.3869934
  z: -1.2103353
  visibility: 

In [8]:
#Keypoints Extrations to (1,-1) array
def extract_keypoints(res) :
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    left = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3) 
    right = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,left,right])

#### DATA COLLECTING

In [9]:
#Def Save Folder
LEARN_DATA_PATH = os.path.join('LN_DATA')
#Def ACTION
actions = np.array(['NO ACTION','ROTATION','PUSH'])
#Thirty videos for all action
no_sequences = 30 #50
#Thirty Frame for one video
sequences_length = 30 #50


In [ ]:
for action in actions :
    for sequence in range(no_sequences) :
        try:
            os.makedirs(os.path.join(LEARN_DATA_PATH,action,str(sequence)))
        except:
            pass

In [23]:
#Collecting landMark for Own DATASET
cap = cv2.VideoCapture(0)
#Declaration of holistic model 
with mp_holistic.Holistic(min_detection_confidence = 0.5 ,min_tracking_confidence = 0.5 ) as holistic:
    
    #LoOp for acting
    for action in actions:
        #LoOP for sequences
        for sequence in range(no_sequences):
            #LoOp for Frames : 30 FRAMES
            for frame_num in range(sequences_length):
                
            
                ret,frame = cap.read()                    # Frame Capture

                image , results = mediapipe_detection(frame,holistic) # Detect LandMArk

                drawing_spec_landmarks(image,results) # Draw Landmarks
                
                
                #Frame recovery coordination 
                if frame_num == 0:
                    cv2.putText(image,"COMMENCER LA COLLECTION",(120,200),
                               cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    
                    cv2.putText(image,'Do {}  {}'.format(action,sequence),(90,200),
                               cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4,cv2.LINE_AA)
                    cv2.imshow("FRAME FEED ", image)          #Display 
                    cv2.waitKey(2000)
                    
                else:
                    cv2.putText(image,'for {}  {}'.format(action,sequence),(900,200),
                               cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4,cv2.LINE_AA)
                    cv2.imshow("FRAME FEED ", image)          #Display
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(LEARN_DATA_PATH,action,str(sequence),str(frame_num))
                np.save(npy_path,keypoints)

                

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

In [18]:
#KILL CV PROCESS    
cap.release()
cv2.destroyAllWindows()

##### DATA SPLIT

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [11]:
#Mapping ACTION
label_map = {label:num for num,label in enumerate(actions)}

In [12]:
label_map

{'NO ACTION': 0, 'ROTATION': 1, 'PUSH': 2}

In [27]:
#CONSTRUCTION OF OUR DATASET
sequences,labels = [],[]
for action in actions :
    for sequence in range(no_sequences):
        window =[] 
        for frame_num in range(sequences_length):
            res =np.load(os.path.join(LEARN_DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [28]:
X = np.array(sequences)

In [29]:
y = to_categorical(labels).astype(int) #Label one hot encoding

In [30]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.05)

#### TRAINING MODEL

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard


In [14]:
log_dir =os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)


In [15]:
#Def own model
model = Sequential()
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662)))
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))

In [34]:
model.compile(optimizer='Adam',loss="categorical_crossentropy",metrics=['categorical_accuracy'])

In [36]:
#Train 
model.fit(X_train,y_train,epochs=350,callbacks=[tb_callback])

Epoch 1/350
3/3 [==============================] - 2s 135ms/step - loss: 7.2278 - categorical_accuracy: 0.3176
Epoch 2/350
3/3 [==============================] - 0s 132ms/step - loss: 6.4805 - categorical_accuracy: 0.3765
Epoch 3/350
3/3 [==============================] - 0s 137ms/step - loss: 5.7421 - categorical_accuracy: 0.3059
Epoch 4/350
3/3 [==============================] - 0s 133ms/step - loss: 8.0138 - categorical_accuracy: 0.3882
Epoch 5/350
3/3 [==============================] - 0s 135ms/step - loss: 16.7623 - categorical_accuracy: 0.3059
Epoch 6/350
3/3 [==============================] - 0s 138ms/step - loss: 15.5635 - categorical_accuracy: 0.3647
Epoch 7/350
3/3 [==============================] - 0s 140ms/step - loss: 12.4564 - categorical_accuracy: 0.4000
Epoch 8/350
3/3 [==============================] - 0s 134ms/step - loss: 12.6690 - categorical_accuracy: 0.4000
Epoch 9/350
3/3 [==============================] - 0s 127ms/step - loss: 9.0233 - categorical_accuracy: 0.38

3/3 [==============================] - 0s 134ms/step - loss: 8.3551 - categorical_accuracy: 0.3294
Epoch 75/350
3/3 [==============================] - 0s 133ms/step - loss: 8.0178 - categorical_accuracy: 0.2706
Epoch 76/350
3/3 [==============================] - 0s 135ms/step - loss: 7.2705 - categorical_accuracy: 0.3529
Epoch 77/350
3/3 [==============================] - 0s 133ms/step - loss: 5.1315 - categorical_accuracy: 0.3059
Epoch 78/350
3/3 [==============================] - 0s 138ms/step - loss: 3.5625 - categorical_accuracy: 0.3529
Epoch 79/350
3/3 [==============================] - 0s 138ms/step - loss: 3.3022 - categorical_accuracy: 0.3765
Epoch 80/350
3/3 [==============================] - 0s 134ms/step - loss: 6.0496 - categorical_accuracy: 0.2353
Epoch 81/350
3/3 [==============================] - 0s 135ms/step - loss: 4.6407 - categorical_accuracy: 0.4000
Epoch 82/350
3/3 [==============================] - 0s 136ms/step - loss: 4.6028 - categorical_accuracy: 0.3529
Epoch

3/3 [==============================] - 0s 136ms/step - loss: 1.9499 - categorical_accuracy: 0.4353
Epoch 219/350
3/3 [==============================] - 0s 131ms/step - loss: 2.4110 - categorical_accuracy: 0.3412
Epoch 220/350
3/3 [==============================] - 0s 133ms/step - loss: 1.8924 - categorical_accuracy: 0.4000
Epoch 221/350
3/3 [==============================] - 1s 239ms/step - loss: 1.6896 - categorical_accuracy: 0.4000
Epoch 222/350
3/3 [==============================] - 0s 156ms/step - loss: 1.4162 - categorical_accuracy: 0.5176
Epoch 223/350
3/3 [==============================] - 0s 160ms/step - loss: 1.4668 - categorical_accuracy: 0.4235
Epoch 224/350
3/3 [==============================] - 0s 144ms/step - loss: 1.4559 - categorical_accuracy: 0.4706
Epoch 225/350
3/3 [==============================] - 0s 133ms/step - loss: 2.2069 - categorical_accuracy: 0.3176
Epoch 226/350
3/3 [==============================] - 0s 133ms/step - loss: 1.5872 - categorical_accuracy: 0.38

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

In [37]:
res = model.predict(X_test)

1/1 [==============================] - 2s 2s/step


In [39]:
actions[np.argmax(res[4])]

'NO ACT'

In [40]:
y_test

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0]])

In [44]:
#model.save("actions.h5")

In [16]:
model.load_weights('actions.h5')

In [17]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score

In [42]:
y_predicts = model.predict(X_train)

3/3 [==============================] - 0s 42ms/step


In [43]:
y_true = np.argmax(y_train,axis=1).tolist()
y_predicts = np.argmax(y_predicts,axis=1).tolist()

In [44]:
multilabel_confusion_matrix(y_true,y_predicts)

array([[[25, 33],
        [ 3, 24]],

       [[56,  0],
        [26,  3]],

       [[40, 16],
        [20,  9]]], dtype=int64)

In [45]:
accuracy_score(y_true,y_predicts)

0.4235294117647059

In [18]:
#Def PROBA VISUALIZER 
colors = [(245,117,36),(117,245,36),(16,117,245)]
def proba_visual(res,actions,input_frame,colors):
    output_frame = input_frame.copy()
    for num,proba in enumerate(res):
        cv2.rectangle(output_frame,(0,60+num*40),(int(proba*100),90+num*40),colors[num],-1)
        cv2.putText(output_frame,actions[num],(0, 85+num*40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    
    return output_frame

#### REAL TIME DETECTION AND PREDICTIONS

In [20]:
#Variables de Detections
sequence = []
suite_of_action = []
predictions = []
threshold = 0.5


cap = cv2.VideoCapture(0)
#Init holistic model 
with mp_holistic.Holistic(min_detection_confidence = 0.5 ,min_tracking_confidence = 0.5 ) as holistic:
    
    while cap.isOpened() :
        ret,frame = cap.read()                   

        

        image , results = mediapipe_detection(frame,holistic)
        
        
        
        drawing_spec_landmarks(image,results)
        
        #  Prediction Logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence,axis =0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            """
            HERE WE PLAN TO ADD 
            - BASKET AVATAR ACTION UPDATE FONCTION WITH UNITY AND C++
            - AND DRONE COMMAND 
            

            """
            
            #visualisation Logic 
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)]> threshold :
                    if len(suite_of_action) > 0:
                        if actions[np.argmax(res)] !=suite_of_action[-1]:
                            suite_of_action.append(actions[np.argmax(res)])
                    else :
                        suite_of_action.append(actions[np.argmax(res)])

            if len(suite_of_action) > 5 :
                    suite_of_action = suite_of_action[-5:]
                
            image = proba_visual(res,actions,image,colors) 
                
        cv2.rectangle(image,(0,0),(700,40),(205,0,0),-1)
        cv2.putText(image,'  '.join(suite_of_action),(3,30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
        
        
    

                
        
        cv2.imshow("FRAME FEED ", image)          

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 51ms/step
ROTATION
1/1 [==============================] - 0s 38ms/step
NO ACTION
1/1 [==============================] - 0s 36ms/step
NO ACTION
1/1 [==============================] - 0s 28ms/step
NO ACTION
1/1 [==============================] - 0s 28ms/step
NO ACTION
1/1 [==============================] - 0s 30ms/step
NO ACTION
1/1 [==============================] - 0s 29ms/step
NO ACTION
1/1 [==============================] - 0s 25ms/step
NO ACTION
1/1 [==============================] - 0s 40ms/step
NO ACTION
1/1 [==============================] - 0s 31ms/step
NO ACTION
1/1 [==============================] - 0s 32ms/step
ROTATION
1/1 [==============================] - 0s 27ms/step
ROTATION
1/1 [==============================] - 0s 28ms/step
ROTATION
1/1 [==============================] - 0s 26ms/step
ROTATION
1/1 [==============================] - 0s 27ms/step
PUSH
1/1 [==============================] - 0s 26ms/step
PUSH
1/1 [==================

1/1 [==============================] - 0s 27ms/step
PUSH
1/1 [==============================] - 0s 25ms/step
PUSH
1/1 [==============================] - 0s 27ms/step
PUSH
1/1 [==============================] - 0s 29ms/step
PUSH
1/1 [==============================] - 0s 26ms/step
PUSH
1/1 [==============================] - 0s 27ms/step
PUSH
1/1 [==============================] - 0s 25ms/step
PUSH
1/1 [==============================] - 0s 24ms/step
PUSH
1/1 [==============================] - 0s 33ms/step
PUSH
1/1 [==============================] - 0s 27ms/step
PUSH
1/1 [==============================] - 0s 27ms/step
PUSH
1/1 [==============================] - 0s 26ms/step
PUSH
1/1 [==============================] - 0s 26ms/step
ROTATION
1/1 [==============================] - 0s 26ms/step
ROTATION
1/1 [==============================] - 0s 36ms/step
ROTATION
1/1 [==============================] - 0s 26ms/step
ROTATION
1/1 [==============================] - 0s 25ms/step
ROTATION
1/1 [======

In [45]:
print(len(sequence[0]))


1662
